In [6]:
#!pip install requests
import requests
import json
import pandas as pd
import numpy as np
from copy import Error
from datetime import datetime

from google.colab import drive
drive.mount('/content/drive')

def float_to_string(f):
  f = np.round(f,4)
  s = str(f)[2:]
  while len(s) < 4:
    s += "0"
  return s

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Collect betting data

In [2]:
odds = pd.read_csv('https://raw.githubusercontent.com/d141/epl_predictions/main/NHL_Odds_Data/nhl%20odds%2018.csv')
odds_team_names = list(odds.Team.unique())
odds.head()

,Date,Rot,VH,Team,1st,2nd,3rd,Final,Open,Close,Puck Line,Unnamed: 11,Open OU,Unnamed: 13,Close OU,Unnamed: 15
0,1003,1,V,Montreal,1,1,0,2,184,210,1.5,-125,6.0,-110,6.0,-110
1,1003,2,H,Toronto,1,1,0,3,-220,-240,-1.5,105,6.0,-110,6.0,-110
2,1003,3,V,Boston,0,0,0,0,115,105,1.5,-290,5.5,-110,5.5,-120
3,1003,4,H,Washington,2,4,1,7,-135,-115,-1.5,230,5.5,-110,5.5,100
4,1003,5,V,Calgary,0,0,2,2,-130,-130,-1.5,200,5.5,-110,5.5,-115


In [11]:
def process_odds_data(df):
  cols = ['date', 'home_team', 'away_team', 'first_period_home', 'second_period_home', 'third_period_home',
                              'first_period_away', 'second_period_away', 'third_period_away', 'final_score',
                              'opening_moneyline_home', 'closing_moneyline_home', 'opening_moneyline_away', 'closing_moneyline_away',
                              'spread', 'away_spread_odds', 'home_spread_odds', 'open_OU', 'close_OU', 'over_odds', 'under_odds']
  final = pd.DataFrame(columns = cols)

  df.index = np.arange(1, len(df)+1)

  for index, row in df.iterrows():

    game_number = 1
    if index % 2 == 0:
      result = {}
      result['date'] = row['Date']
      result['game_number'] = game_number
      game_number += 1
      result['home_team'] = row['Team']
      result['away_team'] = df.loc[index-1]['Team']
      result['first_period_home'] = row['1st']
      result['second_period_home'] = row['2nd']
      result['third_period_home'] = row['3rd']
      result['first_period_away'] = df.loc[index-1]['1st']
      result['second_period_away'] = df.loc[index-1]['2nd']
      result['third_period_away'] = df.loc[index-1]['3rd']      
      result['final_score'] = row['Final']
      result['opening_moneyline_home'] = row['Open']
      result['opening_moneyline_away'] = df.loc[index-1]['Open']
      result['closing_moneyline_home'] = row['Close']
      result['closing_moneyline_away'] = df.loc[index-1]['Close']
      result['spread'] = row['Puck Line']
      result['away_spread_odds'] = df.loc[index-1]['Unnamed: 11']
      result['home_spread_odds'] = row['Unnamed: 11']
      result['open_OU'] = row['Open OU']
      result['close_OU'] = row['Close OU']
      result['over_odds'] = row['Unnamed: 13']
      result['under_odds'] = row['Unnamed: 13']
      #print(row['Team'], df.loc[index-1]['Team'])
      #print(result)
      final= final.append(result, ignore_index=True)
  #df = pd.concat(games)
  return final
df = process_odds_data(odds)
df.head()

,date,home_team,away_team,first_period_home,second_period_home,third_period_home,first_period_away,second_period_away,third_period_away,final_score,opening_moneyline_home,closing_moneyline_home,opening_moneyline_away,closing_moneyline_away,spread,away_spread_odds,home_spread_odds,open_OU,close_OU,over_odds,under_odds,game_number
0,1003,Toronto,Montreal,1,1,0,1,1,0,3,-220,-240,184,210,-1.5,-125,105,6.0,6.0,-110,-110,1.0
1,1003,Washington,Boston,2,4,1,0,0,0,7,-135,-115,115,105,-1.5,-290,230,5.5,5.5,-110,-110,1.0
2,1003,Vancouver,Calgary,1,0,4,0,0,2,5,110,120,-130,-130,1.5,200,-240,5.5,5.5,-110,-110,1.0
3,1003,SanJose,Anaheim,1,1,0,1,1,3,2,-170,-180,148,160,-1.5,-185,165,5.5,5.5,-110,-110,1.0
4,1004,Pittsburgh,Washington,2,3,1,3,1,2,7,-160,-160,140,145,-1.5,-200,170,6.0,6.0,-110,-110,1.0


# Collect game history

In [62]:
def float_to_string(f):
  f = np.round(f,4)
  s = str(f)[2:]
  while len(s) < 4:
    s += "0"
  return s

end_reached = False
year='2019'
game_number = .0001
list_2018 = []

while not end_reached:

  try:
    game_string = float_to_string(game_number)
    boxscore_string = f"https://statsapi.web.nhl.com/api/v1/game/{year}02{game_string}/boxscore"
    boxscore_response = requests.get(boxscore_string)
  except:
    #Reached the end of the season
    end_reached = True
    break

  game = json.loads(boxscore_response.content.decode('utf-8'))
  teams=['away', 'home']

  for team in teams:
    try:
      players = list(game['teams'][team]['players'].keys())
    except:
      end_reached = True
      print(game_number, game_string)
      break
    stats = []
    for player in players:
      try:
        game['teams'][team]['players'][player]['stats']['skaterStats']['player_id'] = player
        stats.append(game['teams'][team]['players'][player]['stats']['skaterStats'])
      except:
        try:
          #Player is a goalie or is scratched
          game['teams'][team]['players'][player]['stats']['goalieStats']['player_id'] = player
          stats.append(game['teams'][team]['players'][player]['stats']['goalieStats'])
        except:
          #Player scratched
          continue
    df = pd.DataFrame.from_dict(stats)
    df = df.set_index('player_id')
    df[f'{team}_team_name'] = game['teams'][team]['team']['name']
    df[f'{team}_team_id'] = game['teams'][team]['team']['id']
    df['season'] = year
    df['game_number'] = game_number
    list_2018.append(df)
  game_number += .0001

In [ ]:
df_2018 = pd.concat(list_2018)

In [ ]:
df_2018.sample(10)

In [ ]:
len(df_2018)

# Process game data

In [4]:
from copy import Error
def process_box_scores(df):

  df['game_number'] = df['game_number'] * 10000
  df['game_number'] = df['game_number'].astype(int)

  home_teams = df_2018[df_2018['away_team_name'].isna()]
  away_teams = df_2018[df_2018['home_team_name'].isna()]
  by_game_away = away_teams.groupby(['game_number']).sum()
  by_game_home = home_teams.groupby(['game_number']).sum()

  num_games = by_game_away.index.argmax()

  if num_games != by_game_home.index.argmax():
    print("Error: Number of home and away games don't match")
    return False

  season = []

  for i in range(1,num_games+1,1):
    box_score = {}
    this_game_all = df[df['game_number'] == i]
    this_game_home = this_game_all[this_game_all['away_team_name'].isna()]
    this_game_away = this_game_all[this_game_all['home_team_name'].isna()]
    box_score['game_number'] = i
    try:
      box_score['home_team_name'] = this_game_home.home_team_name.iloc[0]
      box_score['away_team_name'] = this_game_away.away_team_name.iloc[0]
    except:
      print(f"Error getting the team names in game {i}")
    #print(by_game_home)
    try:
      box_score['home_goals'] = this_game_home.goals.sum()
      box_score['away_goals'] = this_game_away.goals.sum()
      box_score['home_shots'] = this_game_home[this_game_home.evenSaves.isna()].shots.sum()
      box_score['away_shots'] = this_game_away[this_game_away.evenSaves.isna()].shots.sum()
      box_score['home_hits'] = this_game_home.hits.sum()
      box_score['away_hits'] = this_game_away.hits.sum()
      box_score['home_ppg'] = this_game_home.powerPlayGoals.sum()
      box_score['away_ppg'] = this_game_away.powerPlayGoals.sum()
      box_score['home_goals'] = this_game_home.goals.sum()
      box_score['away_goals'] = this_game_away.goals.sum()
    except:
      print("Error getting goals")
      print()
      return False
    season.append(box_score)

  return pd.DataFrame(season)
  
def get_linescore_data(df, year):
  df['start_time'] = ""
  df['periods_played'] = 0
  for i in range(len(df)):
    game_string = float_to_string((df.loc[i].game_number)/10000)
    linescore_string = f"https://statsapi.web.nhl.com/api/v1/game/{year}02{game_string}/linescore"
    linescore_response = requests.get(linescore_string)
    linescore = json.loads(linescore_response.content.decode('utf-8'))

    num_periods = int(linescore['currentPeriod'])

    #for i in range(num_periods):
    #  period = linescore['periods'][0]
    #  period_string = f"{period + 1}"
    #  df[f""]

    df.at[i, 'start_time'] = datetime.strptime(linescore['periods'][0]['startTime'], "%Y-%m-%dT%H:%M:%Sz")
    df.at[i, 'periods_played'] = num_periods
  return df

In [7]:
df_2018 = pd.read_csv('/content/drive/MyDrive/2018_box_scores.csv')
new_df = process_box_scores(df_2018)
df2 = get_linescore_data(new_df, "2018")


Error getting the team names in game 24
Error getting the team names in game 48
Error getting the team names in game 93
Error getting the team names in game 356


In [9]:
df2.to_csv('2018 with linescore')

In [10]:
df2.head()

,game_number,home_team_name,away_team_name,home_goals,away_goals,home_shots,away_shots,home_hits,away_hits,home_ppg,away_ppg,start_time,periods_played
0,1,Toronto Maple Leafs,Montréal Canadiens,3,2,26,36,18.0,34.0,1.0,1.0,2018-10-03 23:16:14,4
1,2,Washington Capitals,Boston Bruins,7,0,37,25,16.0,28.0,4.0,0.0,2018-10-03 23:50:10,3
2,3,Vancouver Canucks,Calgary Flames,5,2,23,35,16.0,17.0,0.0,0.0,2018-10-04 02:11:59,3
3,4,San Jose Sharks,Anaheim Ducks,2,5,33,15,21.0,25.0,0.0,2.0,2018-10-04 02:35:12,3
4,5,Buffalo Sabres,Boston Bruins,0,4,32,26,25.0,13.0,0.0,1.0,2018-10-04 23:10:46,3
